## Import libraries

## Import libraries

In [1]:
import pandas as pd
import requests
import string
string.punctuation
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import numpy as np
from datetime import timezone
import datetime
import time
import pandas as pd
import requests
import string
string.punctuation
import re
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
import contractions

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from nltk import word_tokenize   
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.dates as mdate



from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from datetime import timezone
import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Create Function to retrieve dataset from reddit

Start from 1st April 2021 and later. Take from PTSD subreddit --> time_from = 1617235200

In [4]:
# Create function that takes in search name and limit for search and time

def get_data(title_name, data_min, column, time_from = 1617235200):
    # initiate date to search
    date = time_from

    #instatiate main df for PTSD
    df = pd.DataFrame(columns = column)


    while df.shape[0]<data_min:
        url = 'https://api.pushshift.io/reddit/search/submission'
        params ={
            'subreddit': title_name,
            'before': date,
            'size': 70
        }
        # create temporary df 
        data_temp = pd.DataFrame(requests.get(url,params).json()['data'])[column]

        # create list for index with [removed], '', [deleted] and NAN values
        index_df = list(
            data_temp[
                (data_temp['selftext'] == '[removed]')
                | (data_temp['selftext'] == '')
                | (data_temp['selftext'] == '[deleted]')
                | (data_temp['selftext'].isnull())
                
            ].index)

        # drop down all in index list 
        data_temp = data_temp.drop(index = index_df)
        data_temp.drop_duplicates(subset=['selftext', 'title'], keep='last', inplace=True)
        # concat with main df
        df = pd.concat([df,data_temp])

        # print to check on shape 
        print(df.shape)

        # set up new date for searching after 
        date = df.iloc[-1,-1]
        
    return df

To draw out CPTSD subreddit

In [5]:
col = ['author', 'subreddit', 'selftext', 'title', 'created_utc']

# creating df for PTSD
df_ptsd = get_data('CPTSD', 10000, col)

# Create another column for datetime 
df_ptsd['datetime'] = df_ptsd['created_utc'].apply(lambda x: pd.to_datetime(x, unit='s'))

(54, 5)
(114, 5)
(174, 5)
(233, 5)
(300, 5)
(360, 5)
(421, 5)
(483, 5)
(550, 5)
(616, 5)
(684, 5)
(746, 5)
(806, 5)
(871, 5)
(938, 5)
(1007, 5)
(1072, 5)
(1141, 5)
(1211, 5)
(1273, 5)
(1341, 5)
(1411, 5)
(1479, 5)
(1547, 5)
(1615, 5)
(1682, 5)
(1742, 5)
(1794, 5)
(1852, 5)
(1911, 5)
(1959, 5)
(2028, 5)
(2084, 5)
(2140, 5)
(2202, 5)
(2260, 5)
(2309, 5)
(2369, 5)
(2428, 5)
(2490, 5)
(2543, 5)
(2595, 5)
(2658, 5)
(2725, 5)
(2795, 5)
(2863, 5)
(2933, 5)
(2999, 5)
(3065, 5)
(3133, 5)
(3200, 5)
(3267, 5)
(3333, 5)
(3401, 5)
(3464, 5)
(3530, 5)
(3595, 5)
(3660, 5)
(3728, 5)
(3794, 5)
(3860, 5)
(3930, 5)
(3993, 5)
(4062, 5)
(4128, 5)
(4192, 5)
(4259, 5)
(4323, 5)
(4387, 5)
(4455, 5)
(4508, 5)
(4569, 5)
(4627, 5)
(4678, 5)
(4733, 5)
(4792, 5)
(4843, 5)
(4895, 5)
(4953, 5)
(5010, 5)
(5072, 5)
(5139, 5)
(5208, 5)
(5272, 5)
(5336, 5)
(5404, 5)
(5470, 5)
(5536, 5)
(5602, 5)
(5669, 5)
(5738, 5)
(5802, 5)
(5867, 5)
(5933, 5)
(6000, 5)
(6067, 5)
(6135, 5)
(6202, 5)
(6270, 5)
(6336, 5)
(6404, 5)
(6469,

## Pre-Processing

Combining selftext and title into alltext

In [6]:
# create all text column for self text and title combined 

df_ptsd['alltext'] = df_ptsd['selftext'] + ' ' + df_ptsd['title']

Calculate length of text and title

In [7]:
# to create calculate length of text and title 

df_ptsd['length_text'] = df_ptsd['alltext'].map(lambda x: len(x))

Word count per post

In [8]:
# to create calculate status word cout of text and title 
tokenizer = RegexpTokenizer('\s+', gaps=True)

def tokenizer_func(text):
    return len(tokenizer.tokenize(text))

df_ptsd['wrdcount_text'] = df_ptsd['alltext'].apply(lambda x: tokenizer_func(x))

Check for null values

In [9]:
# found NAN coming from the link filter
df_ptsd.isnull().sum()

author           0
subreddit        0
selftext         0
title            0
created_utc      0
datetime         0
alltext          0
length_text      0
wrdcount_text    0
dtype: int64

I will now **create the target variable** to be used in classification modelling. From this point forward in all classification models, the **positive class (1) will be *r/PTSD*** and the **negative class (0) will be *r/Anxiety***.

In [10]:
# change Ptsd to 1 and Anxiety to 0

df_ptsd['subreddit'] = df_ptsd['subreddit'].map(lambda x: 1 if x == 'ptsd' else 0)

Create datetime object into another column

In [11]:
# Converting 'created_utc' column from epoch time to datetime object.

df_ptsd['datetime'] = df_ptsd['created_utc'].map(lambda ts: datetime.datetime.fromtimestamp(ts))

In [12]:
def change_month(month):
    convert = month.to_pydatetime().month
    month_dict = {
        1: 'Jan',
        2: 'Feb',
        3: 'Mar',
        4: 'April',
        5: 'May',
        6: 'June',
        7: 'July',
        8: "Aug",
        9: 'Sept',
        10: 'Oct',
        11: 'Nov',
        12: 'Dec'
    }

    return month_dict[convert]

df_ptsd['month'] = df_ptsd['datetime'].apply(lambda x: x.to_pydatetime().month)

In [13]:
df_ptsd['day'] = df_ptsd['datetime'].apply(lambda x: x.strftime('%A'))

### Cleaning text - Without Lemmatizing/stemming

I will be defining a function that helps to do the following: 

- **Remove HTML artefacts** using *BeautifulSoup* library.
- **Expand all contractions** using *contractions* library.
- **Remove all numbers, punctuations and special characters, except '-'** to keep hyphenated words, using *re* library.
- **Convert all text to lowercase**.
- **Tokenize all words (hyphenated words stay hyphenated)** using *RegexpTokenizer* from *nltk.tokenize*.
- **Remove all stopwords** using the english stopwords list from *nltk.corpus*.
- **Remove subreddit names** *('PTSD', 'Anxiety')* and other related words to avoid **target leakage**.
- **Join all tokenized words** into a string separated by spaces.

Stemming/Lemmatizing of words will be done during error analysis. 

In [14]:
# Defining a function to clean all_text column following the steps mentioned above.
def clean_text(text_raw):
    
    # Removing any and all HTML artefacts from the text (just in case reddit API still left behind some).
    text_html_removed = BeautifulSoup(text_raw).get_text()
    
    # Expanding all contractions using the contractions library (for example isn't is converted to is not).
    text_contrac_fixed = contractions.fix(text_html_removed)
    
    # Using regex to remove all numbers, punctuations and special characters (except '-' to keep hyphenated words).
    text_punc_removed = re.sub(r'[^A-Za-z\-\s]', ' ', text_contrac_fixed)
    
    # Tokenizing the text after converting it to lowercase.
    # Regex here tokenizes by every character except one or more ('+') consecutive word characters ('\w') or hyphens ('-').
    text_tokens_list = RegexpTokenizer(r'[\w\-]+').tokenize(text_punc_removed.lower())
    
    # Removing stopwords using the english stopwords list from nltk.corpus.
    text_stopwords_removed = [word for word in text_tokens_list if word not in set(nltk.corpus.stopwords.words('english'))]
    
    # Removing subreddit name from text (target leakage).
    text_cleaned = [word for word in text_stopwords_removed if word not in set(['ptsd', 'anxious', 'anxiety',"cptsd", 'trauma','traumatized','traumas',"posttraumatic",'stress','disorder',"traumatic"])]
    
    # Returning a string of all tokenized words joined together with spaces in between.
    return " ".join(text_cleaned)

In [15]:
# apply function to clean text
df_ptsd['clean_text']= df_ptsd['alltext'].apply(lambda x: clean_text(x)) 

In [16]:
# Instantiate lemmatizer.
lemmatizer = WordNetLemmatizer()

# define function that takes in words from paragraph, and lemmitize and join back as paragraph
def lemmi_words(text):
    return ' '.join([lemmatizer.lemmatize(i) for i in text.split(' ')])

df_ptsd['lemmi_clean_text'] = df_ptsd['clean_text'].map(lambda x: lemmi_words(x))

In [17]:
df_export = df_ptsd[['author','subreddit','lemmi_clean_text']]

In [18]:
# record csv
df_export.to_csv('../Data/TEST_UNSEEN.csv')